In [1]:
import numpy as np
import cv2
import dlib
import matplotlib.pyplot as plt

# Các hàm xử lý

In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
SOURCE = './dataset/messi.jpg'

In [4]:
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index

In [5]:
def constructLandmarks(img_gray):
    faces = detector(img_gray)
    for face in faces:
        landmarks = predictor(img_gray, face)
        landmarks_points = []
        
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmarks_points.append((x, y))


        points = np.array(landmarks_points, np.int32)
        convexhull = cv2.convexHull(points)
                
    return points, landmarks_points, convexhull

In [6]:
def exportTriangles(convexhull, points, landmarks_points):
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks_points)
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32)

    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])


        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)

        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)

        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)

        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3]
            indexes_triangles.append(triangle)
    return indexes_triangles

In [7]:
def affineTransform(indexes_triangles, img, img2_new_face, source_landmark_points, points1, dest_landmark_points, points2):
    for triangle_index in indexes_triangles:
        
        # Triangulation of the first face
        tr1_pt1 = source_landmark_points[triangle_index[0]]
        tr1_pt2 = source_landmark_points[triangle_index[1]]
        tr1_pt3 = source_landmark_points[triangle_index[2]]
        triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)


        rect1 = cv2.boundingRect(triangle1)
        (x, y, w, h) = rect1
        cropped_triangle = img[y: y + h, x: x + w]
        cropped_tr1_mask = np.zeros((h, w), np.uint8)


        points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                           [tr1_pt2[0] - x, tr1_pt2[1] - y],
                           [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)

        cv2.fillConvexPoly(cropped_tr1_mask, points, 255)

        # Lines space
        cv2.line(lines_space_mask, tr1_pt1, tr1_pt2, 255)
        cv2.line(lines_space_mask, tr1_pt2, tr1_pt3, 255)
        cv2.line(lines_space_mask, tr1_pt1, tr1_pt3, 255)
        lines_space = cv2.bitwise_and(img, img, mask=lines_space_mask)

        # Triangulation of second face
        tr2_pt1 = dest_landmark_points[triangle_index[0]]
        tr2_pt2 = dest_landmark_points[triangle_index[1]]
        tr2_pt3 = dest_landmark_points[triangle_index[2]]
        triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)


        rect2 = cv2.boundingRect(triangle2)
        (x, y, w, h) = rect2

        cropped_tr2_mask = np.zeros((h, w), np.uint8)

        points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                            [tr2_pt2[0] - x, tr2_pt2[1] - y],
                            [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)

        cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)

        # Warp triangles
        points = np.float32(points)
        points2 = np.float32(points2)
        M = cv2.getAffineTransform(points, points2)
        warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
        warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_tr2_mask)

        # Reconstructing destination face
        img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
        img2_new_face_rect_area_gray = cv2.cvtColor(img2_new_face_rect_area, cv2.COLOR_BGR2GRAY)
        _, mask_triangles_designed = cv2.threshold(img2_new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
        warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

        img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
        img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area
    return img2_new_face

In [8]:
def match(img2_gray, img2, convexhull2, img2_new_face):
    img2_face_mask = np.zeros_like(img2_gray)
    img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
    img2_face_mask = cv2.bitwise_not(img2_head_mask)


    img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
    result = cv2.add(img2_head_noface, img2_new_face)

    (x, y, w, h) = cv2.boundingRect(convexhull2)
    center_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))

    seamlessclone = cv2.seamlessClone(result, img2, img2_head_mask, center_face2, cv2.NORMAL_CLONE)
    return seamlessclone

# Chuẩn bị source

In [9]:
source_img = cv2.imread(SOURCE)
source_gray = cv2.cvtColor(source_img, cv2.COLOR_BGR2GRAY)
source_mask = np.zeros_like(source_gray)

In [10]:
source_img_copy = np.copy(source_img)
source_gray_copy = np.copy(source_gray)
source_mask_copy = np.copy(source_mask)

In [11]:
src_points, src_landmark_points, src_convexhull = constructLandmarks(source_gray)
lines_space_mask = np.zeros_like(source_gray_copy)

In [12]:
index_triangles = exportTriangles(src_convexhull, src_points, src_landmark_points)

In [13]:
input_video_path = './Mark-vid-input.mp4'
cap = cv2.VideoCapture(input_video_path)
# cap = cv2.VideoCapture(0)

In [14]:
width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = round(cap.get(cv2.CAP_PROP_FPS))

In [15]:
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter("ResultSwapVid.mp4", fourcc, fps, (width, height))

In [16]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Img, gray, mask for dest
        dest_img = frame
        dest_gray = cv2.cvtColor(dest_img, cv2.COLOR_BGR2GRAY)
        dest_mask = np.zeros_like(source_gray)
        height, width, channels = dest_img.shape
        dest_new_face = np.zeros((height, width, channels), np.uint8)
        
        
        # Copy for dest
        dest_img_copy = np.copy(dest_img)
        dest_gray_copy = np.copy(dest_gray)
        dest_mask_copy = np.copy(dest_mask)
        lines_space_new_face = np.zeros_like(dest_img_copy)
        
        # Get params for dest
        dest_points, dest_landmark_points, dest_convexhull = constructLandmarks(dest_gray_copy)
        
        # Transform
        dest_new_face = affineTransform(index_triangles, source_img_copy, dest_new_face, src_landmark_points, src_points, dest_landmark_points, dest_points)
        
        # Swap and match
        result = match(dest_gray_copy, dest_img_copy, dest_convexhull, dest_new_face)
        
        out.write(result)
        cv2.imshow("frame", result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()